In [235]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from multiprocessing import Process, Queue
import os
import numpy as np
import seaborn as sns
import tensorflow as tf
import pandas as pd
from scipy.misc import imsave
import scipy.stats
import random
from sklearn.metrics import accuracy_score

from problog import get_evaluatable
from problog.program import PrologString, SimpleProgram
from problog.core import ProbLog
from problog.logic import Constant,Var,Term,AnnotatedDisjunction
from problog.learning import lfi

%matplotlib notebook

In [283]:
model = """
t(0.5)::gender_male;t(0.5)::gender_female.
t(0.25)::age_1;t(0.25)::age_2;t(0.25)::age_3;t(0.25)::age_4.
t(0.5)::smoke.
t(0.5)::alcohol.

t(0.33)::bmi_1;t(0.33)::bmi_2;t(0.33)::bmi_3.
t(0.5)::ldl.
t(0.33)::blood_pressure_1;t(0.33)::blood_pressure_2;t(0.33)::blood_pressure_3.

t(_)::blood_pressure_1:-gender_male.
t(_)::blood_pressure_1:-gender_female.
t(_)::blood_pressure_1:-age_1.
t(_)::blood_pressure_1:-age_2.
t(_)::blood_pressure_1:-age_3.
t(_)::blood_pressure_1:-age_4.
t(_)::blood_pressure_1:-smoke.
t(_)::blood_pressure_1:-alcohol.

t(_)::blood_pressure_2:-gender_male.
t(_)::blood_pressure_2:-gender_female.
t(_)::blood_pressure_2:-age_1.
t(_)::blood_pressure_2:-age_2.
t(_)::blood_pressure_2:-age_3.
t(_)::blood_pressure_2:-age_4.
t(_)::blood_pressure_2:-smoke.
t(_)::blood_pressure_2:-alcohol.

t(_)::blood_pressure_3:-gender_male.
t(_)::blood_pressure_3:-gender_female.
t(_)::blood_pressure_3:-age_1.
t(_)::blood_pressure_3:-age_2.
t(_)::blood_pressure_3:-age_3.
t(_)::blood_pressure_3:-age_4.
t(_)::blood_pressure_3:-smoke.
t(_)::blood_pressure_3:-alcohol.

t(_)::ldl_t:-gender_male.
t(_)::ldl_t:-gender_female.
t(_)::ldl_t:-age_1.
t(_)::ldl_t:-age_2.
t(_)::ldl_t:-age_3.
t(_)::ldl_t:-age_4.
t(_)::ldl_t:-smoke.
t(_)::ldl_t:-alcohol.

t(_)::ldl_f:-gender_male.
t(_)::ldl_f:-gender_female.
t(_)::ldl_f:-age_1.
t(_)::ldl_f:-age_2.
t(_)::ldl_f:-age_3.
t(_)::ldl_f:-age_4.
t(_)::ldl_f:-smoke.
t(_)::ldl_f:-alcohol.

t(_)::bmi_1:-gender_male.
t(_)::bmi_1:-gender_female.
t(_)::bmi_1:-age_1.
t(_)::bmi_1:-age_2.
t(_)::bmi_1:-age_3.
t(_)::bmi_1:-age_4.
t(_)::bmi_1:-smoke.
t(_)::bmi_1:-alcohol.

t(_)::bmi_2:-gender_male.
t(_)::bmi_2:-gender_female.
t(_)::bmi_2:-age_1.
t(_)::bmi_2:-age_2.
t(_)::bmi_2:-age_3.
t(_)::bmi_2:-age_4.
t(_)::bmi_2:-smoke.
t(_)::bmi_2:-alcohol.

t(_)::bmi_3:-gender_male.
t(_)::bmi_3:-gender_female.
t(_)::bmi_3:-age_1.
t(_)::bmi_3:-age_2.
t(_)::bmi_3:-age_3.
t(_)::bmi_3:-age_4.
t(_)::bmi_3:-smoke.
t(_)::bmi_3:-alcohol.

t(_)::heart_d_t:-blood_pressure_1.
t(_)::heart_d_t:-blood_pressure_2.
t(_)::heart_d_t:-blood_pressure_3.
t(_)::heart_d_t:-ldl.
t(_)::heart_d_t:-bmi_1.
t(_)::heart_d_t:-bmi_2.
t(_)::heart_d_t:-bmi_3.

t(_)::heart_d_f:-blood_pressure_1.
t(_)::heart_d_f:-blood_pressure_2.
t(_)::heart_d_f:-blood_pressure_3.
t(_)::heart_d_f:-ldl.
t(_)::heart_d_f:-bmi_1.
t(_)::heart_d_f:-bmi_2.
t(_)::heart_d_f:-bmi_3.

t(_)::glucose_t:-blood_pressure_1.
t(_)::glucose_t:-blood_pressure_2.
t(_)::glucose_t:-blood_pressure_3.
t(_)::glucose_t:-ldl.
t(_)::glucose_t:-bmi_1.
t(_)::glucose_t:-bmi_2.
t(_)::glucose_t:-bmi_3.

t(_)::glucose_f:-blood_pressure_1.
t(_)::glucose_f:-blood_pressure_2.
t(_)::glucose_f:-blood_pressure_3.
t(_)::glucose_f:-ldl.
t(_)::glucose_f:-bmi_1.
t(_)::glucose_f:-bmi_2.
t(_)::glucose_f:-bmi_3.
"""

In [284]:
gender_male = Term('gender_male')
gender_female = Term('gender_female')

age_1 = Term('age_1')
age_2 = Term('age_2')
age_3 = Term('age_3')
age_4 = Term('age_4')

smoke_t = Term('smoke')
smoke_f = Term('smoke')

alcohol_t = Term('alcohol')
alcohol_f = Term('alcohol')

bmi_1 = Term('bmi_1')
bmi_2 = Term('bmi_2')
bmi_3 = Term('bmi_3')

ldl_t = Term('ldl')
ldl_f = Term('ldl')

blood_pressure_1 = Term('blood_pressure_1')
blood_pressure_2 = Term('blood_pressure_2')
blood_pressure_3 = Term('blood_pressure_3')

heart_d = Term('heart_d')

glucose = Term('glucose')

In [285]:
examples = [
    [(gender_female, True), (age_2, True), (blood_pressure_1, True)],
    [(gender_female, False), (age_2, False), (blood_pressure_2, False)]
]

In [286]:
score, weights, atoms, iteration, lfi_problem = lfi.run_lfi(PrologString(model), examples)

In [287]:
print (lfi_problem.get_model())

0.5::gender_male; 0.5::gender_female.
0.0::age_1; 0.5::age_2; 0.499999482066499::age_3; 0.0::age_4.
0.602672926951633::smoke_t; 0.397327073048374::smoke_f.
0.431150913052601::alcohol_t; 0.568849086947414::alcohol_f.
0.33::bmi_1; 0.33::bmi_2; 0.33::bmi_3.
0.5::ldl_t; 0.5::ldl_f.
0.577901930494966::blood_pressure_1; 0.0::blood_pressure_2; 0.33::blood_pressure_3.
0.256330823154478::blood_pressure_1 :- gender_male.
0.876698280635917::blood_pressure_1 :- gender_female.
0.113990994764195::blood_pressure_1 :- age_1.
0.9999997937158::blood_pressure_1 :- age_2.
0.290180739547995::blood_pressure_1 :- age_3.
0.150323220551294::blood_pressure_1 :- age_4.
0.444131012134975::blood_pressure_1 :- smoke_t.
0.513645719375475::blood_pressure_1 :- smoke_f.
0.112486166298793::blood_pressure_1 :- alcohol_t.
0.30652476622531::blood_pressure_1 :- alcohol_f.
0.0::blood_pressure_2 :- gender_male.
0.012649427084679::blood_pressure_2 :- gender_female.
0.978062316103723::blood_pressure_2 :- age_1.
0.32734207555116